# Estimating mean and variance through back propagation.

Although deep learning typically solves classification problems, it is still a powerful approach
regression problems. As any entry level ML course teaches you, the loss function for regression
is MSE (mean squared error). 

One of the core reasons MSE is the right loss is because it assumes the distribution of errors follows
a normal distribution, and that the variance (or standard deviation) of those errors is constant. The former is a good assumption because the CLT (central limit theorem) sort of says that the "average" distribution is the normal distribution. 

Looking at the PDF (probability distribution function) of a Normal distribution

$$
f(x \mid \mu, \sigma^2) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left(-\frac{(x - \mu)^2}{2\sigma^2}\right)
$$

we see hidden in there the square of the delta between the mean and a datapoint: ${(x - \mu)^2}$. That's actually the heart of why we use the MSE: when estimating the mean $\mu$, when we see a datapoint $x$, we consider that an Error, then we Square it, then we just take the Mean of all the datapoints. That's how we end up using MSE. (Note that $x$ in this case represents the target value, which is a label in your dataset and typically called y_true in the context of training the parameters of a neural network, and mu is the neural network's estimate of the mean of the y value, called y_pred or y_hat)

Why can we ignore the variance or $\sigma^2$? Because an assumption is that the variance of the error is constant. That's called "homoscedastic" - homo meaning the same like in homonym. So when we calculate the gradient, we can ignore any constant value. 

What about that exponent function? Well, training neural networks is typically based on MLE (Maximum Likelihood Estimation), which says that whatever parameters give you the highest joint probability matching the output are the most likely params. Joint probability involves taking the product of lots of small numbers - for numerical stability you can log the whole thing to make it a sum of logs instead, that's one reason we find the log function in the classic crossentropy loss used for classification.

$$
\text{Cross-Entropy Loss} = - \left[ y \cdot \log(\hat{y}) + (1 - y) \cdot \log(1 - \hat{y}) \right]
$$

It's also why MSE doesn't have any pesky exponent functions - a log is applied to it, and the log and exponent basically cancel each other out. 

If the variance is not constant then the data has "heteroscedasticity" - hetero means different, like in heterogeneous, and we actually run into lots of problems when trying to do things like t-tests, but that's not the focus of this article.

The focus is how to get a back-propped model to estimate variance. The answer is simply to output two heads - one for mean of our y value, one for variance, and undo the two shortcuts we applied above (constant variance, log of exponent). Based on MLE, we still need the NLL (negative of the log of the likelihood). And the likelihood is mathematically the same thing as the probability according to the frequentistic statistics behind MLE. And the probability is given by the PDF. 

Basically, all we have to do is the $-\log(\text{pdf})$, where $\text{pdf}$ is a function estimating the probability of the prediction being x given the parameters mean and variance. The form you'll see this in ML textbooks will be something like the following, which means the loss function is the log of the pdf of an example value 

$$
-\log(\text{pdf}(y_true | \theta))
$$

where $\theta$ represents the predictions of your neural network, namely, mean and variance. It's really that simple.

And just to be explicit: -log(pdf) simplifies to MSE when the variance is constant. That's why the use of MSE is the direct outcome of applying MLE when variance is constant. 

Some of the first ideas here came from econometrics, where they wanted to estimate time-series data but realizes that during some periods of time, there is higher variance than others. This [book](https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=db869fa192a3222ae4f2d766674a378e47013b1b) talks about it further in the context of ML, and of course we have to go to Fisher for MLE and Gauss and Laplace for CLT. 

## First, setup some data with different variances. 

We'll train a model to estimate the expected value and variance of the output of a function, based on the input. 

We'll have two clusters of data. If the input is between zero and one, the output is a normal distribution with mean 1 and stdev 1 (var 1).
If the input is between ten and eleven, the output is a normal distribution with mean -1 and stdev 2 (var 4)

In [128]:
import torch
import torch.nn as nn

x1 = torch.randint(0, 1, (10000, 1)).float()
y1 = torch.randn(10000, 1) + 1

x2 = torch.randint(10, 11, (10000, 1)).float()
y2 = torch.randn(10000, 1) * 2 -1

x = torch.cat([x1, x2])
y = torch.cat([y1, y2])

### Now let's setup a data loader for that data. 

In [129]:
class Data(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

dataloader = torch.utils.data.DataLoader(Data(x, y), batch_size=100, shuffle=True, drop_last=True)

## Now setup a model.

Nothing fancy here. Just an MLP (multilayer perceptron) with two heads. 

In [165]:
class Predictor(nn.Module):
    def __init__(self):
        super(Predictor, self).__init__()

        hidden_depth = 20

        self.dense1 = nn.Linear(1, hidden_depth)
        self.tanh1 = nn.Tanh()
        self.dense2 = nn.Linear(hidden_depth, hidden_depth)
        self.tanh2 = nn.Tanh()
        self.dense3 = nn.Linear(hidden_depth, hidden_depth)
        self.tanh3 = nn.Tanh()
        self.dense4 = nn.Linear(hidden_depth, hidden_depth)
        self.tanh4 = nn.Tanh()
        self.dense5 = nn.Linear(hidden_depth, hidden_depth)

        self.mean_y = nn.Linear(hidden_depth, 1)
        self.var_y = nn.Linear(hidden_depth, 1)
    
    def forward(self, x):
        x = self.dense1(x)
        x = self.tanh1(x)
        x = self.dense2(x)
        x = self.tanh2(x)
        x = self.dense3(x)
        x = self.tanh3(x)
        x = self.dense4(x)
        x = self.tanh4(x)
        x = self.dense5(x)


        mean = self.mean_y(x)
        var = torch.exp(self.var_y(x)) # var should be positive
        return mean, var

model = Predictor()

for p in model.parameters():
    p.register_hook(lambda grad: torch.clamp(grad, -1.0, +1.0))




# Let's setup that fancy loss function

Again, it's not fancy at all, it's just NLL applied to both mean and variance of the output. 

In [168]:
class MeanVarianceLoss(nn.Module):
    """Calculates the negative log likelihood of seeing a target value given a mean and a variance."""
    def __init__(self):
        super(MeanVarianceLoss, self).__init__()
    
    def forward(self, mean, var, target):
        normal = torch.distributions.Normal(mean, torch.sqrt(var))

        # For numerical stability, the torch distributions library
        # returns the log(pdf(target | mean, variance)), not the pdf(target | mean, variance) directly.
        log_prob = normal.log_prob(target)

        # NLL is the negative log probability
        nll = -log_prob

        return torch.mean(nll)

criterion = MeanVarianceLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1000.)

from tqdm import tqdm

for epoch in range(100):
    if 0 <= epoch < 20:
        optimizer.param_groups[0]['lr'] = 1e-3
    elif 20 <= epoch < 40:
        optimizer.param_groups[0]['lr'] = 1e-4
    elif 40 <= epoch < 60:
        optimizer.param_groups[0]['lr'] = 1e-5
    elif 60 <= epoch < 80:
        optimizer.param_groups[0]['lr'] = 1e-6
    else:
        optimizer.param_groups[0]['lr'] = 1e-8

    for x, y in dataloader:
        optimizer.zero_grad()
        mean, var = model(x)
        loss = criterion(mean, var, y)
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0: 
        print(f"Epoch {epoch}, Loss: {loss.item()}")

Epoch 0, Loss: 1.8573625087738037
Epoch 10, Loss: 1.7618536949157715
Epoch 20, Loss: 1.8004021644592285
Epoch 30, Loss: 1.920356273651123
Epoch 40, Loss: 1.7423737049102783
Epoch 50, Loss: 1.7630614042282104
Epoch 60, Loss: 1.6256223917007446
Epoch 70, Loss: 1.7487053871154785
Epoch 80, Loss: 1.7681242227554321
Epoch 90, Loss: 1.7784992456436157


## Did it work? Let's spot check it's estimate of mean and stdev on some numbers from our two sets.     

In [170]:
mean, var = model(torch.tensor([0.5]))
print(f"Estimate of mean and var for first cluster of data should be mean=1, var=1.")
print(f"Mean estimate: {mean.item()}, Var estimate: {var.item()}")
print()

mean, var = model(torch.tensor([10.5]))
print(f"Estimate of mean and var for second cluster of data should be mean=-1, var=4.")
print(f"Mean estimate: {mean.item()}, Var estimate: {var.item()}")

Estimate of mean and var for first cluster of data should be mean=1, var=1.
Mean estimate: 0.6582908034324646, Var estimate: 1.3874123096466064

Estimate of mean and var for second cluster of data should be mean=-1, var=4.
Mean estimate: -1.0008701086044312, Var estimate: 3.9981565475463867
